# Datenbankverbindungen mit Python

 In diesem Notebook lernen wir, wie wir: 
 * eine Verbindung zur Datenbank herstellen
 * SQL Ausführen (und Parameter aus Python nutzen)
 
Die Methoden die für den Zugriff auf die Datenbank nötig sind, sind in der Python Database API Specification v2 [PEP 249](https://www.python.org/dev/peps/pep-0249/) beschrieben.



## Eine Verbindung zur Datenbank herstellen

Die Datenbankverbindung wird über das Modul der Datenbank aufgebaut. Dieses Modul verfügt über eine `connect( parameters... )`-Methode. Die genauen Parameter, die eine Datenbank für die Verbindung benötigt, kann jedes Datenbank-Modul selbst festlegen. 

In unseren Beispiel verwenden wir eine Sqlite3-Datenbank. Der Parameter `:memory:` erzeugt eine Datenbank im Hauptspeicher. Diese Datenbank ist sehr schnell erstellt, aber dafür nicht persistent. Zum erproben der Verbindungsfunktionen ist dies jedoch vollkommen ausreichend. 

In [2]:
%run Datenbankzugriff_Util.ipynb # import zu Überprüfung der Übungen

# Verbindung mit der Datenbank herstellen
import sqlite3
conn = sqlite3.connect(':memory:')


#import mysql.connector
#conn = mysql.connector.connect(
#  host="localhost",
#  user="yourusername",
#  password="yourpassword",
#  database="mydatabase"
#)

## SQL Ausführen

SQL-Statements werden mithilfe eines Cursors ausgeführt. Ein Cursor kann auch DML- oder DDL-Anfragen ausführen. Bei DQL-Abfragen können Datensätze mit fetchall() direkt geladen werden. 

In [21]:

cur = conn.cursor()
cur.execute('''CREATE TABLE emp(id NUMBER, name VARCHAR(255), sal NUMBER)''')
cur.close()

c= conn.cursor()
# Daten einfuegen
c.execute("INSERT INTO emp VALUES (1, 'Smith', 3200)")
c.execute("INSERT INTO emp VALUES (2, ?, 2800)", [("Adams")])
c.executemany("INSERT INTO emp VALUES (?, ?, ?)", 
                [(3, "King", 5200), (4,"Johnson", 3400)])
conn.commit();

# Abfrage von Daten (SELECT)

c.execute('SELECT * FROM emp')
result = c.fetchall()
for x in result:
  print(x[1])
c.close()
  
  
print("Der Zugriff über den Spaltennamen ist auch möglich")
conn.row_factory = sqlite3.Row    
c= conn.cursor()
c.execute('SELECT * FROM emp')
result = c.fetchall()
for x in result:
  print(x['name'])
c.close()   
    
    
# fetchone bis None
print("Für größere Datenmengen sollte man auf fetchall aus performencegründen verzichten. ")
c= conn.cursor()
c.execute('SELECT * FROM emp')
row = c.fetchone()
while row != None:
  print(row['name'])
  row = c.fetchone()

c.close()

Smith
Adams
King
Johnson
Der Zugriff über den Spaltennamen ist auch möglich
Smith
Adams
King
Johnson
Für größere Datenmengen sollte man auch fetchall aus performencegründen verzichten. 
Smith
Adams
King
Johnson


# Übung: Datensätze einfügen

Fügen Sie folgenden Datensatz in die Tabelle emp der Datenbank ein.

| ID   | Name       | Salary | 
|------|------------|--------|
| 10   | May        | 4100   | 

In [1]:
#TODO: hier Datensatz einfügen



check(conn)

NameError: name 'check' is not defined

## Fehlerbehandlung

In [69]:
from sqlite3 import OperationalError

cur = conn.cursor()
try:
  cur.execute('SELECT * FROM non_existing_emp')
except  OperationalError as e:
    print("Bei der Datenbankabfrage ist ein Fehler aufgetreten: " + str(e));
cur.close();

Bei der Datenbankabfrage ist ein Fehler aufgetreten: no such table: non_existing_emp


## Integration von Python in SQLite

Da die Datenbankengine SQLite komplett in Python integriert ist, lassen sich Python-Funktionen direkt in SQL nutzen. 
Um eine normale Single-Row-Funktion zu Verwenden muss man diese in der Verbindung mit `create_function` registieren.
Dieser Mechanismus funktioniert auch mit Multi-Row-Funktionen. Eine Multi-Row-Funktion wird in Python durch eine Klasse abgebildet. 

In [9]:
#
# Single Row Function
#
def format_ename(t):
    return "Emp Name: " + t 


conn.create_function("format_ename", 1, format_ename)
cur = conn.cursor()
cur.execute("select format_ename(e.name) from emp e")
print(cur.fetchone()[0])

#
# Das funktioniert auch mit Aggregatsfunktionen
#
class SqrSum:
    def __init__(self):
        self.count = 0

    def step(self, value):
        self.count += value*value

    def finalize(self):
        return self.count

conn.create_aggregate("sqr_sum", 1, SqrSum)
cur = conn.cursor()
cur.execute("select sqr_sum(sal) from emp")
print("Die quadartische Summe der Gehälter ist " + str(cur.fetchone()[0]))


Emp Name: Smith
Die quadartische Summe der Gehälter ist 56680000
